<a href="https://colab.research.google.com/github/Kyungmin-Yu/BTC_prediction/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## mecab 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/final_title.csv', encoding='CP949')

In [ ]:
data = data.loc[(data['date']>'2017-12-31'),]

In [ ]:
date_list = np.unique(data['date']).tolist()

In [ ]:
document_list = []
for i in range(len(date_list)):
  date = date_list[i]
  titles = data.loc[data.date==date,'title'].tolist()
  document = " ".join(titles)
  document_list.append(document)

In [ ]:
import re
final_document=[]
for n in document_list:
    hangul1 = re.compile('[^ ㄱ-ㅣ가-힣]+')
    hangul2 = hangul1.sub('', n)
    hangul3 = hangul2.replace("로 변경시 확장됨", "")
    result = hangul3.replace(r"  ", " ")
    final_document.append(result)

In [ ]:
## tokenizer 함수
from konlpy.tag import Mecab
def tokenizer(raw, pos=["NNG", "NNB", "VV", "VA", "VX", "VCP", "VCN", "IC", "NNP", "EF"]):
  m = Mecab()
  return [word for word, tag in m.pos(raw) if len(word) > 1 and tag in pos ]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(tokenizer=tokenizer, max_features=300)
dtm = tfidf_vect.fit_transform(final_document)

In [ ]:
btc_data = pd.read_csv('/content/drive/MyDrive/data/final_data.csv')

In [ ]:
close = btc_data.loc[(btc_data['date']>'2017-12-31')&(btc_data['date']<'2021-05-01'),['btc', 'date']]

In [ ]:
close['yesterday'] = close['btc'].shift(1)

In [ ]:
close['diff'] = close['btc'] - close['yesterday']

In [ ]:
close = close.assign(
    UP = lambda df: df['diff'].map(lambda diff: 1 if diff>0 else 0)
)

In [ ]:
up_idx = close[close['UP']>0].index.tolist()
down_idx = close[close['UP']==0].index.tolist()

In [ ]:
y = close['UP'][1:]
X = dtm[:1215,:].toarray()

In [ ]:
from sklearn.linear_model import Ridge
model = Ridge(solver="sag", fit_intercept=True, random_state=369)
model.fit(X,y)
beta = list(model.coef_)

In [ ]:
k = list(tfidf_vect.vocabulary_.keys())
sentiment_dict = dict(zip(k, beta))

In [ ]:
idx_list = []
for i in range(dtm.toarray().shape[0]):
  idx_list.append(list(dtm.toarray()[i]>0))

In [ ]:
sentiment_score = []
for k in range(dtm.toarray().shape[0]):
  score = np.sum([beta[i] for i in [i for i,idx in enumerate(idx_list[k]) if idx==True]])
  sentiment_score.append(score)

In [ ]:
btc_data['sentiment'] = sentiment_score
btc_data.to_csv('/content/drive/MyDrive/data/real_final_data2.csv', index=False)